In [5]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [6]:
import time
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, DataCollatorWithPadding, AutoModel
from tqdm import tqdm
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from torchmetrics.classification import MultilabelAccuracy
import pickle
import random


In [3]:
from huggingface_hub import notebook_login

notebook_login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
# train_df = pd.read_csv('/kaggle/input/dataset/train.csv')
# test_df = pd.read_csv('/kaggle/input/dataset/test.csv')
train_df.head()

,id,comment,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
0,0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,0,0,0,0,0,0,0,0,0,0,0,0
1,1,บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแ...,0,0,0,0,0,0,0,0,0,0,0,0
2,2,เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซน...,0,0,0,0,0,0,0,0,0,0,0,0
3,3,ปัญหาน้ำท่วมในซอยสวนพลู 1,0,0,0,0,0,0,0,0,0,0,0,0
4,4,1. ซ่อมสายไฟ กรีดขวางทางเท้า,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
train_df.describe()

,id,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
count,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000,204622.000000
mean,102310.500000,0.209635,0.019969,0.000049,0.016034,0.000117,0.167944,0.034542,0.000406,0.024176,0.018478,0.000059,0.023839
std,59069.427728,0.407049,0.139892,0.006991,0.125608,0.010829,0.373817,0.182616,0.020136,0.153597,0.134672,0.007658,0.152548
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51155.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,102310.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,153465.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,204621.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
def drop_rows(df):
    # drop rows with missing values, duplicates
    print(f"Before dropna: {df.shape}")
    df.dropna(inplace=True)
    df.drop_duplicates("comment", keep="first", inplace=True)
    df = df.reset_index(drop=True)
    print(f"After dropna: {df.shape}")

drop_rows(train_df)
drop_rows(test_df)
print(train_df.shape, test_df.shape)

Before dropna: (204622, 14)
After dropna: (204622, 14)
Before dropna: (10810, 2)
After dropna: (10810, 2)
(204622, 14) (10810, 2)


In [8]:
contacts = train_df.columns[2:]

train_df['label'] = train_df[contacts].apply(lambda x: list(x), axis=1)
train_df.drop(contacts, axis=1, inplace=True)
train_df.drop("id", axis=1, inplace=True)

test_df.drop("id", axis=1, inplace=True)
train_df.head()

,comment,label
0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,บริเวณสะพานสามถนนจันทน์ เป็นจุดเปลี่ยนถ่ายสองแ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,เรื่องทางม้าลายหายไป บริเวณสี่แยกถนนจันทร์-เซน...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,ปัญหาน้ำท่วมในซอยสวนพลู 1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,1. ซ่อมสายไฟ กรีดขวางทางเท้า,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [9]:
train_df['label'].value_counts()

label
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    102333
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]     42212
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]     32543
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]      5803
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]      4598
                                         ...  
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]         1
[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]         1
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]         1
[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]         1
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]         1
Name: count, Length: 73, dtype: int64

In [ ]:
train_df = train_df.loc[train_df['label'].apply(lambda x: sum(x) != 0)]
train_df.reset_index(drop=True, inplace=True)
print(train_df.shape)
train_df['label'].value_counts()

(102289, 2)


label
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    42212
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]    32543
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]     5803
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]     4598
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]     3993
                                        ...  
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]        1
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]        1
[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]        1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]        1
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]        1
Name: count, Length: 72, dtype: int64

In [32]:
contacts

Index(['สำนักงานตำรวจแห่งชาติ', 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
       'สภาเด็กและเยาวชนกรุงเทพมหานคร', 'กรมควบคุมมลพิษ', 'กรมสรรพสามิต',
       'การไฟฟ้านครหลวง', 'กรมทางหลวง', 'สำนักงานประกันสุขภาพแห่งชาติ',
       'การประปานครหลวง', 'คณะกรรมการการพัฒนาเศรษฐกิจ',
       'กระทรวงการท่องเที่ยวและกีฬา', 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200'],
      dtype='object')

In [33]:
train_data = train_df.copy().to_numpy()
test_data = test_df.copy().to_numpy()

label_2_idx = dict(zip(contacts, range(len(contacts))))
idx_2_label = dict(zip(range(len(contacts)), contacts))

print("Create Mappings")
display(idx_2_label)
display(label_2_idx)

# print("Before Mappings")
# display(data[:, 1])
# data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'สำนักงานตำรวจแห่งชาติ',
 1: 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
 2: 'สภาเด็กและเยาวชนกรุงเทพมหานคร',
 3: 'กรมควบคุมมลพิษ',
 4: 'กรมสรรพสามิต',
 5: 'การไฟฟ้านครหลวง',
 6: 'กรมทางหลวง',
 7: 'สำนักงานประกันสุขภาพแห่งชาติ',
 8: 'การประปานครหลวง',
 9: 'คณะกรรมการการพัฒนาเศรษฐกิจ',
 10: 'กระทรวงการท่องเที่ยวและกีฬา',
 11: 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200'}

{'สำนักงานตำรวจแห่งชาติ': 0,
 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย': 1,
 'สภาเด็กและเยาวชนกรุงเทพมหานคร': 2,
 'กรมควบคุมมลพิษ': 3,
 'กรมสรรพสามิต': 4,
 'การไฟฟ้านครหลวง': 5,
 'กรมทางหลวง': 6,
 'สำนักงานประกันสุขภาพแห่งชาติ': 7,
 'การประปานครหลวง': 8,
 'คณะกรรมการการพัฒนาเศรษฐกิจ': 9,
 'กระทรวงการท่องเที่ยวและกีฬา': 10,
 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200': 11}

In [34]:
def set_random_seed(seed: int):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(42)

In [41]:
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", model_max_length=512, revision='main')
# tokenizer = Tokenizer("airesearch/wangchanberta-base-att-spm-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

ValueError: Converting from Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast convertors: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'RealmTokenizer', 'Refor

In [6]:
tokenizer(train_data[0,0])

NameError: name 'train_data' is not defined

In [16]:
for i, sample in tqdm(enumerate(train_data)):
    cleaned_sentence = sample[0].replace('ํา', "ำ") # only cleaning needed for WangchanBERTa
    train_data[i, 0] = tokenizer(cleaned_sentence) # no padding as we will use DataCollatorWithPadding to pad within batch

204622it [01:18, 2611.62it/s]


In [17]:
for i, sample in tqdm(enumerate(test_data)):
    cleaned_sentence = sample[0].replace('ํา', "ำ") # only cleaning needed for WangchanBERTa
    test_data[i, 0] = tokenizer(cleaned_sentence) # no padding as we will use DataCollatorWithPadding to pad within batch

10810it [00:04, 2272.48it/s]


In [7]:
with open('train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)
with open('test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

NameError: name 'train_data' is not defined

In [28]:
with open('/kaggle/input/pickle/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('/kaggle/input/pickle/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [29]:
X_train = train_data[:, 0]
y_train = train_data[:, 1]
X_test = test_data[:, 0]
y_test = np.array([([0] * 12) + [1] for _ in range(test_data.shape[0])])

for y in y_train:
    if y == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]:
        y.append(1)
    else:
        y.append(0)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((204622,), (204622,), (10810,), (10810, 13))

In [30]:
print(X_train[0])
print(y_train[0])

print(X_test[0])
print(y_test[0])

{'input_ids': [5, 10, 385, 3755, 8, 8, 10, 818, 501, 13080, 8, 10, 991, 12681, 488, 13773, 537, 8, 10, 75, 1548, 6161, 82, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
{'input_ids': [5, 10, 3029, 1548, 39, 445, 32, 1981, 24925, 28, 317, 618, 1433, 9, 5099, 9, 2649, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[0 0 0 0 0 0 0 0 0 0 0 0 1]


In [31]:
class TraffyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, device='cpu'):
        self.encodings = encodings
        self.labels = labels
        self.device = device

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(val[:512]).to(self.device) for key, val in self.encodings[idx].items()
            if key in ['input_ids', 'attention_mask'] # take only input_ids and attention_mask fields
        }
        item['labels'] = torch.tensor(self.labels[idx]).to(self.device)
        return item

    def __len__(self):
        return len(self.labels)

In [32]:
train_dataset = TraffyDataset(X_train, y_train, device='cuda')
test_dataset = TraffyDataset(X_test, y_test, device='cuda')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=data_collator)

In [33]:
train_loader.dataset[0]['input_ids']

tensor([    5,    10,   385,  3755,     8,     8,    10,   818,   501, 13080,
            8,    10,   991, 12681,   488, 13773,   537,     8,    10,    75,
         1548,  6161,    82,     6], device='cuda:0')

In [34]:
print(train_loader.dataset[0])
print(test_loader.dataset[0]['labels'])
print(train_loader.dataset[0]['labels'].float().dtype)

{'input_ids': tensor([    5,    10,   385,  3755,     8,     8,    10,   818,   501, 13080,
            8,    10,   991, 12681,   488, 13773,   537,     8,    10,    75,
         1548,  6161,    82,     6], device='cuda:0'), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0'), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')}
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')
torch.float32


In [35]:
class BaseModel(LightningModule):
    def __init__(
          self,
          model_name: str = 'airesearch/wangchanberta-base-att-spm-uncased',
          learning_rate: float = 2e-5
    ):
        super().__init__()
        self.save_hyperparameters()

        self.encoder = AutoModel.from_pretrained(model_name)
        self.learning_rate = learning_rate

    def get_embeddings(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids, attention_mask)
        hidden_states = outputs.last_hidden_state

        # [CLS] token is the first token in the sequence (index 0)
        cls_embeddings = hidden_states[:, 0, :]  # [batch_size, hidden_size]
        
        return cls_embeddings

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

    def forward(self, input_ids, attention_mask):
        return self.get_embeddings(input_ids, attention_mask)

In [36]:
class LMWithMultiLabelClassfier(BaseModel):
    def __init__(
          self,
          model_name: str = 'airesearch/wangchanberta-base-att-spm-uncased',
          ckpt_path: str = None,
          learning_rate: float = 2e-5,
          freeze_encoder_weights: bool = False,
          num_labels: int = 13
    ):
        super().__init__(
            model_name,
            learning_rate
        )
        self.save_hyperparameters()

        if ckpt_path:
            checkpoint = torch.load(ckpt_path)
            encoder_state_dict = {k.replace("encoder.", ""): v for k, v in checkpoint["state_dict"].items() if k.startswith("encoder.")}
            self.encoder.load_state_dict(encoder_state_dict)

        self.linear_layer = nn.Linear(768, num_labels)

        if freeze_encoder_weights:
          self.freeze_weights(self.encoder)  # Freeze model

        self.accuracy = MultilabelAccuracy(num_labels=num_labels, threshold=0.5)

    def freeze_weights(self, model):
        for param in model.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        cls_embeddings = self.get_embeddings(input_ids, attention_mask)
        logits = self.linear_layer(cls_embeddings)
        return logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels'].float()  # float for BCEWithLogitsLoss
        logits = self.forward(input_ids, attention_mask)
        print(f"Logits: {logits}")
        print(f"Labels: {labels}")
        print(f"Logits shape: {logits.shape}")
        print(f"Labels shape: {labels.shape}")

        loss = F.binary_cross_entropy_with_logits(logits, labels)
        print(f"loss: {loss}")
        self.log('train_loss', loss, prog_bar=True)

        preds = torch.sigmoid(logits)
        print(f"preds: {preds}")
        acc = self.accuracy(preds, labels)
        self.log('train_acc', acc, prog_bar=True)

        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels'].float()
        logits = self.forward(input_ids, attention_mask)

        loss = F.binary_cross_entropy_with_logits(logits, labels)
        self.log('test_loss', loss, prog_bar=True)

        preds = torch.sigmoid(logits)
        acc = self.accuracy(preds, labels)
        self.log('test_acc', acc, prog_bar=True)

        return loss

In [37]:
model = LMWithMultiLabelClassfier(
    'airesearch/wangchanberta-base-att-spm-uncased',
    ckpt_path=None,
    freeze_encoder_weights=False
)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="val_acc",  # Metric to monitor
    mode="max",  # "min" for loss, "max" for accuracy
    save_top_k=1,  # Save only the best model(s)
    save_weights_only=True, # Saves only weights, not the entire model
    dirpath="./checkpoints/", # Path where the checkpoints will be saved
    filename="best_pretrained_w_linear_model-{epoch}-{val_acc:.2f}", # Customized name for the checkpoint
    verbose=True,
)

linear_trainer = Trainer(
    max_epochs=5,
    accelerator='auto',
    callbacks=[checkpoint_callback], # Add the ModelCheckpoint callback
    gradient_clip_val=1.0,
    precision="16-mixed", # Mixed precision training
    devices=1,
)

In [39]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'


In [40]:
!export CUDA_LAUNCH_BLOCKING=1
!export TORCH_USE_CUDA_DSA=1

In [41]:
start_time = time.time()
# Train the model
linear_trainer.fit(model, train_loader)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

Training: |          | 0/? [00:00<?, ?it/s]

Logits: tensor([[-0.6641,  0.1681,  0.3337, -0.2815,  0.7729, -0.9214,  0.7051, -0.5391,
         -1.5664, -0.4880,  0.8628, -0.5415,  1.2441],
        [-0.9360,  0.2786, -0.1091, -0.3970,  1.0039, -0.1500,  0.6719, -0.6514,
         -0.6572, -0.2627,  0.6851, -0.8164,  0.9688],
        [-0.1284, -0.4253, -0.6201,  0.0895,  0.2605, -0.5215,  0.8389, -1.0928,
         -0.5200, -0.4348,  1.1836, -0.2195,  0.1559],
        [-0.3086,  0.1232, -0.1296, -0.3853,  0.2644, -0.2664,  0.4434, -0.1658,
         -0.6914, -0.0790,  0.7461,  0.1228,  1.3623],
        [-0.3000, -0.2139,  0.0617,  0.6738,  0.5508, -0.7974,  0.6895, -0.5205,
         -0.2793, -0.6538,  0.4463, -0.8892,  1.0078],
        [-1.2373,  0.1729, -0.6450,  0.5122,  0.6729, -0.6025,  1.3164, -0.0074,
         -1.2666, -0.7368,  1.1172, -1.0254,  1.0830],
        [-0.2822,  0.1558, -0.3228, -0.4836,  0.0550, -0.0098,  1.5020, -0.1030,
         -0.7642, -0.5713,  0.6958, -0.1477,  1.0283],
        [-0.8193,  0.1985, -0.2898, -0.5

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [29]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
start_time = time.time()
result = linear_trainer.test(model, test_loader)
end_time = time.time()

print(f"Total time: {end_time - start_time}")